In [ ]:
%pip install --no-cache-dir "litellm==1.77.4" "openai==1.99.9"  "dspy-ai==3.0.3"
%pip install 'litellm[proxy]' --quiet
%pip install requests --quiet
%pip install -q  wandb==0.22.0 
%pip install -q weave
%pip install -q groq
%pip install -q cerebras_cloud_sdk
%pip install pyyaml --quiet

import os
import sys
import wandb
import logging
import yaml
import subprocess
import time
from pathlib import Path

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logging.info("Logging is now visible!")

# Load secrets
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    os.environ["HF_TOKEN"] = user_secrets.get_secret("HF_TOKEN")
    os.environ["GROQ_API_KEY"] = user_secrets.get_secret("GROQ_API_KEY")
    os.environ["GROQ_API_KEY_153"] = user_secrets.get_secret("GROQ_API_KEY_153")
    os.environ["CEREBRAS_API_KEY"] = user_secrets.get_secret("CEREBRAS_API_KEY")
    os.environ["CEREBRAS_API_KEY_153"] = user_secrets.get_secret("CEREBRAS_API_KEY_153")
    os.environ["OPENROUTER_API_KEY"] = user_secrets.get_secret('OPENROUTER_API_KEY')
    os.environ["OPENROUTER_API_KEY_153"] = user_secrets.get_secret('OPENROUTER_API_KEY_153')
else:
    from google.colab import userdata
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY_153"] = userdata.get('GROQ_API_KEY_153')
    os.environ["CEREBRAS_API_KEY"] = userdata.get('CEREBRAS_API_KEY')
    os.environ["CEREBRAS_API_KEY_153"] = userdata.get('CEREBRAS_API_KEY_153')
    os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
    os.environ["OPENROUTER_API_KEY_153"] = userdata.get('OPENROUTER_API_KEY_153')

# Constants for environment names
KAGGLE = "Kaggle"
COLAB = "Colab"
LOCAL = "Local"

def get_python_major_minor():
    version_tuple = sys.version_info[:2]
    return f"{version_tuple[0]}.{version_tuple[1]}"

def get_environment():
    """
    Determines the current execution environment and retrieves relevant settings.

    Returns:
        tuple: A tuple containing:
            - environment (str): The name of the detected environment.
            - working_dir (str): The working directory path.
            - python_version (str): The version of Python in use.
            - wandb_api_key (str or None): The Wandb API key, if available.
    """
    environment = LOCAL
    working_dir = os.getcwd()
    python_version = sys.version.split()[0]
    wandb_api_key = None

    python_version = get_python_major_minor()

    if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
        try:
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

            environment = KAGGLE
            working_dir = "/kaggle/working/"

        except ImportError:
            print("Error: Unable to import Kaggle secrets module.")
    elif "google.colab" in sys.modules:
        try:
            from google.colab import userdata
            environment = COLAB
            working_dir = "/content"

            wandb_api_key = userdata.get('WANDB_API_KEY')
            if not wandb_api_key:
                print("Warning: WANDB_API_KEY not found in Colab userdata.")
        except ImportError:
            print("Error: Unable to import Colab userdata module.")
    else:
        print("Local environment detected.")

    return (environment, working_dir, python_version, wandb_api_key,)

def display_status(message, status="info"):
    if status == "error":
        logging.error(f"[{status.upper()}] {message}")
    elif status == "warning":
        logging.warning(f"[{status.upper()}] {message}")
    else:
        logging.info(f"[{status.upper()}] {message}")

def setup_and_start_ollama(models_to_pull=None):
    if models_to_pull is None:
        models_to_pull = [
            "hf.co/Qwen/Qwen3-8B-GGUF:Q8_0"
        ]

    print("Installing Ollama... This may take a minute...")
    result = os.system("curl -fsSL https://ollama.com/install.sh | sh 2>/dev/null")
    if result == 0:
        display_status("✅ Ollama installed successfully!", "success")
    else:
        display_status("⚠️ Ollama installation had warnings but may still work", "warning")

    print("Starting Ollama server...")
    os.system("nohup ollama serve > /tmp/ollama_serve_stdout.log 2>/tmp/ollama_serve_stderr.log &")
    time.sleep(5)

    running = os.system("ps aux | grep -E 'ollama serve' | grep -v grep > /dev/null 2>&1")
    if running == 0:
        display_status("✅ Ollama server is running!", "success")
    else:
        display_status("❌ Ollama server failed to start. Check troubleshooting section.", "error")

    for model in models_to_pull:
        display_status(f"📦 Pulling model: {model}")
        start_time = time.time()
        result = os.system(f"ollama pull {model}")
        end_time = time.time()

        if result == 0:
            elapsed = end_time - start_time
            display_status(f"✅ Model '{model}' downloaded in {elapsed/60:.1f} minutes!", "success")
        else:
            display_status(f"❌ Failed to download model '{model}'.", "error")

    print("\n📋 Available models:")
    time.sleep(30)
    os.system("ollama list")

def create_litellm_config(config_path, environment):
    """
    Creates LiteLLM proxy config with your exact models and multiple accounts.
    
    Args:
        config_path: Path to save the config file
        environment: Current environment (KAGGLE, COLAB, or LOCAL)
    """
    
    # Base config with cloud providers
    config_content = """
model_list:
  # OpenRouter models
  - model_name: deepseek-chat-v3
    litellm_params:
      model: openrouter/deepseek/deepseek-chat-v3.1:free
      api_key: os.environ/OPENROUTER_API_KEY
  - model_name: deepseek-chat-v3
    litellm_params:
      model: openrouter/deepseek/deepseek-chat-v3.1:free
      api_key: os.environ/OPENROUTER_API_KEY_153
      
  # Cerebras Account 1
  - model_name: o3-mini
    litellm_params:
      model: cerebras/gpt-oss-120b
      api_key: os.environ/CEREBRAS_API_KEY
  - model_name: qwen-3-235b-a22b-thinking-2507
    litellm_params:
      model: cerebras/qwen-3-235b-a22b-thinking-2507
      api_key: os.environ/CEREBRAS_API_KEY
  - model_name: qwen-3-235b-a22b-instruct-2507
    litellm_params:
      model: cerebras/qwen-3-235b-a22b-instruct-2507
      api_key: os.environ/CEREBRAS_API_KEY
      
  # Cerebras Account 2 (153)
  - model_name: o3-mini_153
    litellm_params:
      model: cerebras/gpt-oss-120b
      api_key: os.environ/CEREBRAS_API_KEY_153
  - model_name: qwen-3-235b-a22b-thinking-2507_153
    litellm_params:
      model: cerebras/qwen-3-235b-a22b-thinking-2507
      api_key: os.environ/CEREBRAS_API_KEY_153
  - model_name: qwen-3-235b-a22b-instruct-2507_153
    litellm_params:
      model: cerebras/qwen-3-235b-a22b-instruct-2507
      api_key: os.environ/CEREBRAS_API_KEY_153
      
  # Groq models
  - model_name: gpt-oss-120b
    litellm_params:
      model: groq/openai/gpt-oss-120b
      api_key: os.environ/GROQ_API_KEY
      api_base: https://api.groq.com/openai/v1
  - model_name: gpt-oss-120b
    litellm_params:
      model: groq/openai/gpt-oss-120b
      api_key: os.environ/GROQ_API_KEY_153
      api_base: https://api.groq.com/openai/v1
  - model_name: llama-3.3-70b-versatile
    litellm_params:
      model: groq/llama-3.3-70b-versatile
      api_key: os.environ/GROQ_API_KEY
      api_base: https://api.groq.com/openai/v1
  - model_name: llama-3.3-70b-versatile
    litellm_params:
      model: groq/llama-3.3-70b-versatile
      api_key: os.environ/GROQ_API_KEY_153
      api_base: https://api.groq.com/openai/v1

litellm_settings:
  num_retries: 3
  request_timeout: 6000
  allowed_fails: 1
  cooldown_time: 30
  drop_params: true
  
router_settings:
  routing_strategy: "usage-based-routing-v2"
  
general_settings:
  port: 4000
  cache: false 
"""

    # Add Ollama models only for LOCAL environment
    if environment == LOCAL:
        ollama_models = """
  # Ollama local models
  - model_name: "qwen3-8b-GGUF-q8_0_local"             
    litellm_params:
      model: "ollama/qwen3-8b-GGUF-q8_0"
      api_base: "http://localhost:11434"
      api_key: ollama
  - model_name: "DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL_local"             
    litellm_params:
      model: "ollama/DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL"
      api_base: "http://localhost:11434"
      api_key: ollama
      num_ctx: 130000
  - model_name: "qwen3-8B-192k-GGUF-IQ4_XS_local"             
    litellm_params:
      model: "ollama/qwen3-8B-192k-Context-6X-Larger-GGUF-IQ4_XS"
      api_base: "http://localhost:11434"
      api_key: ollama
"""
        # Insert Ollama models at the beginning
        lines = config_content.split('\n')
        # Find the line with "model_list:" and insert after it
        for i, line in enumerate(lines):
            if line.strip() == "model_list:":
                lines.insert(i + 1, ollama_models)
                break
        config_content = '\n'.join(lines)
        logging.info("Added Ollama models for LOCAL environment")
    
    with open(config_path, 'w') as f:
        f.write(config_content)
    
    logging.info(f"Config file created at: {config_path}")
    return config_path

def start_litellm_proxy(config_path, port=4000):
    """
    Starts LiteLLM proxy server in background.
    
    Args:
        config_path: Path to config file
        port: Port to run proxy on
    
    Returns:
        subprocess.Popen: Process handle
    """
    cmd = [
        "litellm",
        "--config", config_path,
        "--port", str(port),
        "--detailed_debug"
    ]
    
    logging.info(f"Starting LiteLLM proxy on port {port}...")
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # Wait a bit for server to start
    time.sleep(5)
    
    # Check if process is running
    if process.poll() is None:
        logging.info(f"✅ LiteLLM proxy started successfully on http://0.0.0.0:{port}")
    else:
        stdout, stderr = process.communicate()
        logging.error(f"Failed to start proxy:\n{stderr}")
        raise RuntimeError("LiteLLM proxy failed to start")
    
    return process

def main():
    (env_name, working_dir, python_version, wandb_api_key,) = get_environment()

    if wandb_api_key:
        os.environ["WANDB_API_KEY"] = wandb_api_key
        wandb_api_key_length = len(wandb_api_key)
    else:
        wandb_api_key_length = 0

    os.environ["ENVIRONMENT"] = env_name
    os.environ["WORKING_DIR"] = working_dir
    os.environ["WANDB_ENTITY"] = "rogermt23"
    os.environ["WANDB_PROJECT_NAME"] = "nsgf-paper2code-playground"
    os.environ["WEAVE_PROJECT_NAME"] = "nsgf-paper2code-playground"

    os.environ["OPENAI_BASE_URL"] = "http://0.0.0.0:4000"
    os.environ["OPENAI_API_KEY"] = "litellm"
    
    # Display environment information
    print(f"Environment          : {env_name}")
    print(f"Working Directory    : {working_dir}")
    print(f"Python Version       : {python_version}")
    print(f"Wandb API Key Length : {wandb_api_key_length}")

    setup_and_start_ollama([
        #"hf.co/Qwen/Qwen3-8B-GGUF:Q8_0",
        "hf.co/unsloth/DeepSeek-R1-0528-Qwen3-8B-GGUF:Q8_K_XL"
    ])

    # Create config and start proxy
    config_path = os.path.join(working_dir, "litellm_config.yaml")
    create_litellm_config(config_path, env_name)
    
    proxy_process = start_litellm_proxy(config_path)
    
    return working_dir, python_version, proxy_process

# Run the setup in the notebook
working_dir, python_version, proxy_process = main()

print("\n✅ Setup complete! LiteLLM proxy is running.")
print("Use OpenAI SDK with base_url='http://0.0.0.0:4000' and api_key='litellm'")

# Show available models based on environment
if os.environ["ENVIRONMENT"] == LOCAL:
    print("\n📦 Available models (LOCAL):")
    print("  Local Ollama:")
    print("    - qwen3-8b-GGUF-q8_0_local")
    print("    - DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL_local")
    print("    - qwen3-8B-192k-GGUF-IQ4_XS_local")
    print("  Cloud (with account rotation):")
    print("    - deepseek-chat-v3 (OpenRouter x2)")
    print("    - o3-mini, o3-mini_153 (Cerebras)")
    print("    - qwen-3-235b-a22b-thinking-2507 (x2)")
    print("    - qwen-3-235b-a22b-instruct-2507 (x2)")
    print("    - gpt-oss-120b (Groq x2)")
    print("    - llama-3.3-70b-versatile (Groq x2)")
else:
    print("\n📦 Available models (CLOUD):")
    print("    - deepseek-chat-v3 (OpenRouter x2)")
    print("    - o3-mini, o3-mini_153 (Cerebras)")
    print("    - qwen-3-235b-a22b-thinking-2507 (x2)")
    print("    - qwen-3-235b-a22b-instruct-2507 (x2)")
    print("    - gpt-oss-120b (Groq x2)")
    print("    - llama-3.3-70b-versatile (Groq x2)")

print("\nExample:")
print("""
from openai import OpenAI
client = OpenAI(base_url="http://0.0.0.0:4000", api_key="litellm")

# Auto-rotates through both accounts!
response = client.chat.completions.create(
    model="gpt-oss-120b",
    messages=[{"role": "user", "content": "Hello!"}]
)
print(response.choices[0].message.content)
""")


In [ ]:
%cd $working_dir
# Download NSGF paper PDF using curl (or use wget: replace with !wget -O ...)
!curl -o /content/NSGF.pdf https://arxiv.org/pdf/2401.14069.pdf


%cd $working_dir
!git clone -b feature/litellm-support https://github.com/rogermt/Paper2Code.git
!git ls-remote https://github.com/rogermt/Paper2Code.git refs/heads/feature/litellm-support

%cd Paper2Code
!ls -l $working_dir/Paper2Code/codes
%pip install -r requirements.txt -q
%pip install pydantic==2.11.9 --upgrade

%cd $working_dir/Paper2Code

In [ ]:
# 1) Ask GitHub what SHA it has for the branch (no clone needed)
!echo "remote SHA advertised for feature/litellm-support:"
!git ls-remote https://github.com/rogermt/Paper2Code.git refs/heads/feature/litellm-support

# 2) (Optional) show the SHA you expect (replace if different)
!echo "expected SHA (from your laptop): 1c9e4ac"

# 3) Clone fresh into a new folder to avoid any cache
!rm -rf /kaggle/working/Paper2Code_clone_test
!git clone -b feature/litellm-support https://github.com/rogermt/Paper2Code.git /kaggle/working/Paper2Code_clone_test

# 4) Show what commit the new clone checked out
!echo "clone HEAD (short):"
!/bin/bash -c "cd /kaggle/working/Paper2Code_clone_test && git rev-parse --short HEAD"

# 5) Show the first-level file list and codes folder listing so we can confirm which files are present
!echo "top-level files in clone:"
!/bin/bash -c "ls -la /kaggle/working/Paper2Code_clone_test | sed -n '1,200p'"
!echo "codes/ listing:"
!/bin/bash -c "ls -la /kaggle/working/Paper2Code_clone_test/codes | sed -n '1,200p'"

# 6) Also show the remote commits (last 5) on the remote branch (server-side view)
!echo "recent remote commits (server-side) for origin branch:"
!/bin/bash -c "git ls-remote --refs https://github.com/rogermt/Paper2Code.git | grep feature/litellm-support -n || true"
!/bin/bash -c "git ls-remote https://github.com/rogermt/Paper2Code.git refs/heads/feature/litellm-support || true"

In [ ]:
!ollama cp hf.co/Qwen/Qwen3-8B-GGUF:Q8_0 qwen3-8b-GGUF-q8_0
!ollama cp hf.co/unsloth/DeepSeek-R1-0528-Qwen3-8B-GGUF:Q8_K_XL DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL
!ollama list

In [ ]:
%%writefile $working_dir/Paper2Code/notebooks/test_ollama_openai_proxy.py
import argparse
from openai import OpenAI

def main():
    parser = argparse.ArgumentParser(description="Test Ollama OpenAI proxy with a chat completion.")
    parser.add_argument(
        "--base_url",
        type=str,
        default="http://localhost:11434/v1",
        help="Base URL of the OpenAI-compatible proxy (default: http://localhost:11434/v1)"
    )
    parser.add_argument(
        "--model",
        type=str,
        default="hf.co/unsloth/DeepSeek-R1-0528-Qwen3-8B-GGUF:Q8_K_XL",
        help="Model name to use (default: DeepSeek-R1-0528-Qwen3-8B-GGUF:Q8_K_XL)"
    )
    args = parser.parse_args()

    client = OpenAI(
        base_url=args.base_url,
        api_key="ollama"  # Dummy key for local proxy
    )

    response = client.chat.completions.create(
        model=args.model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Capital city of France."}
        ]
    )

    print(response.choices[0].message.content)

if __name__ == "__main__":
    main()


In [ ]:
%%writefile $working_dir/Paper2Code/notebooks/test_ollama_openai_proxy_dspy.py
import argparse
import dspy

def main():
    parser = argparse.ArgumentParser(description="Test DSPy with Ollama OpenAI proxy.")
    parser.add_argument(
        "--base_url",
        type=str,
        default="http://localhost:11434/v1",
        help="Base URL of the OpenAI-compatible proxy (default: http://localhost:11434/v1)"
    )
    parser.add_argument(
        "--model",
        type=str,
        default="ollama/DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL:latest",
        help="Model name to use (default: DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL:latest)"
    )
    args = parser.parse_args()

    # Your original code, untouched except for args
    lm = dspy.LM(
        model=args.model,
        # api_base=args.base_url,  # Left commented as in your original
        api_key="ollama",
        max_tokens=256,
        temperature=0.2,
        cache=False,
    )

    dspy.settings.configure(lm=lm)

    class Summarize(dspy.Signature):
        """Summarize a passage into one sentence."""
        context: str = dspy.InputField()
        summary: str = dspy.OutputField()

    class Summarizer(dspy.Module):
        def __init__(self):
            super().__init__()
            self.predict = dspy.Predict(Summarize)

        def forward(self, context: str):
            return self.predict(context=context)

    passage = "Ollama exposes an OpenAI-compatible API on port 11434, so DSPy can connect directly without a proxy."
    module = Summarizer()
    pred = module(context=passage)
    print("Summary:", pred.summary)

if __name__ == "__main__":
    main()
  

In [ ]:
%%writefile $working_dir/Paper2Code/notebooks/test_litellm_openai_proxy_dspy.py
import dspy
from typing import List
import tiktoken


# Define get_prefixed_model
def get_prefixed_model(model_name: str) -> str:
    """Map model alias to prefixed model for compatibility."""
    if model_name == "qwen3-8b-GGUF-q8_0_local":
        return "ollama/qwen3-8b-GGUF-q8_0"
    return f"openai/{model_name}"

# Configure DSPy with direct Ollama endpoint
gpt_version = "DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL_local"
prefixed_model = get_prefixed_model(gpt_version)
lm = dspy.LM(
    model=prefixed_model,
    api_base="http://localhost:4000",   # LiteLLM proxy
    api_key="anything",                 # proxy accepts any key if master_key not set
    cache=False
)
dspy.settings.configure(lm=lm)

# Define CodeEvaluation signature
class CodeEvaluation(dspy.Signature):
    """Evaluate code based on paper description. Return a JSON object with 'score' (1-5, based on correctness, completeness, and adherence to the paper's methodology) and 'critique_list' (exactly 3 concise critiques, each <=50 characters, explaining the score). An optional explanation may be included for additional context."""
    paper = dspy.InputField(desc="Description of the paper")
    code = dspy.InputField(desc="Code implementation to evaluate")
    score = dspy.OutputField(desc="Score (1-5)", prefix="Score (1-5):")
    critique_list = dspy.OutputField(
        desc="List of 3 critiques, each <=50 characters",
        prefix="Critique List:"
    )

# Define decorated function to trace DSPy Predict
#@weave.op()
def evaluate_code(paper: str, code: str):
    evaluator = dspy.Predict(CodeEvaluation)
    response = evaluator(paper=paper, code=code)
    return response

# Create a large input (~100,000 tokens)
large_paper = "Abstract: This paper introduces a novel algorithm for graph neural networks. " * 50 # ~70,000 tokens
large_code = """
def graph_neural_network(data):
    import torch
    import torch.nn as nn
    class GNN(nn.Module):
        def __init__(self):
            super(GNN, self).__init__()
            self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
""" * 10  # ~30,000 tokens

# Count tokens
encoding = tiktoken.get_encoding("cl100k_base")
paper_tokens = len(encoding.encode(large_paper, allowed_special={"<|endoftext|>"}))
code_tokens = len(encoding.encode(large_code, allowed_special={"<|endoftext|>"}))
print(f"Paper tokens: {paper_tokens}, Code tokens: {code_tokens}, Total: {paper_tokens + code_tokens}")

# Try to evaluate without chunking
try:
    response = evaluate_code(large_paper, large_code)  # Call decorated function
    print("Score:", response.score)
    print("Critique:", response.critique_list)
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
%%bash
python ${WORKING_DIR}/Paper2Code/notebooks/test_ollama_openai_proxy.py
python ${WORKING_DIR}/Paper2Code/notebooks/test_ollama_openai_proxy_dspy.py
#python ${WORKING_DIR}/Paper2Code/notebooks/test_litellm_openai_proxy_dspy.py

In [ ]:
%cd $working_dir/Paper2Code

In [ ]:
%%bash
mkdir $WORKING_DIR/output
cd $WORKING_DIR/output; wget --spider https://raw.githubusercontent.com/rogermt/playground/master/src/NSGF.json \
  && wget https://raw.githubusercontent.com/rogermt/playground/master/src/NSGF.json
cat ${WORKING_DIR}/output/NSGF.json  | head -20

In [ ]:
import shutil

use_kaggle_resumer = True
import shutil

use_kaggle_resumer = False
if use_kaggle_resumer:
    working_dir = "/kaggle/working"
    kaggle_input_dir = "/kaggle/input"
    output_dir  = f"{working_dir}/output/NSGF"
    kaggle_resume_dir="paper2code-nsgf-paper-output"
    resume_dir = f"{kaggle_input_dir}/{kaggle_resume_dir}/output/NSGF"
    plan_traj_json = f'{output_dir}/planning_trajectories.json'
    plan_traj_full_json = f"{working_dir}/planning_trajectories_full.json"
    print(f"{resume_dir}=>{output_dir}")
    shutil.copytree(resume_dir, output_dir, dirs_exist_ok=True)
    shutil.copy(plan_traj_full_json, plan_traj_json)

In [ ]:
%cd $working_dir/Paper2Code/scripts
!bash run_litellm.sh


In [ ]:
%cd $working_dir/Paper2Code/codes
!python eval_litellm.py \
  --paper_name NSGF \
  --gpt_version "DeepSeek-R1-0528-Qwen3-8B-GGUF-Q8_K_XL_local" \
  --pdf_json_path $working_dir/output/NSGF.json \
  --data_dir $working_dir/Paper2Code/data \
  --output_dir $working_dir/output/NSGF \
  --target_repo_dir $working_dir/output/NSGF_repo \
  --eval_result_dir $working_dir/results \
  --eval_type ref_free \
  --generated_n 1 \
  --papercoder \
  --max_context 120000 \
  --delay 0 \
  --temperature 0.6 \
  --wandb_run_id rwo1law5